In [ ]:
from tqdm import tqdm
import networkx as nx
import geopandas as gpd
import statsmodels.api as sm # recommended import according to the docs
import matplotlib.pyplot as plt
import numpy as np
import copy
from multiprocessing import  Process
import multiprocessing
import csv
from functools import cmp_to_key
import random
import matplotlib as mpl
from matplotlib import cm
import cartopy.crs as ccrs
import InterestingColorfulColor as ICC
import json
from shapely.wkb import dumps as b_dumps, loads as b_loads
from threading import  Thread
import os

In [ ]:
'''download TLE'''
import requests
import os
import h5py
from datetime import datetime
import json


# download the current day's TLE data from the CelesTrak website
def download_TLE_data(constellation_name):
    # h5 file path to save TLE data
    file_path = '/data/chenwei/research3/code/StarPerf_Simulator-release-v2.0/config/TLE_constellation/' + constellation_name + '/tle.h5'
    if not os.path.exists(file_path):
        with h5py.File(file_path, 'a') as file:
            pass
    # TLE data url
    url1 = "https://celestrak.org/NORAD/elements/gp.php?GROUP=" + constellation_name + "&FORMAT=2le"
    url2 = "https://celestrak.org/NORAD/elements/gp.php?GROUP=" + constellation_name + "&FORMAT=json"
    try:
        # send HTTP GET request
        response1 = requests.get(url1)
        # check if the request was successful
        response1.raise_for_status()
        # get web page TLE data and split the str by '\n'
        TLE = response1.text.split('\n')
        # delete the last row because it is a blank row with no data
        TLE.pop()

        response2 = requests.get(url2)
        if response2.status_code == 200:
            json_TLE = response2.json()
            # convert each JSON object to a string
            json_TLE = [json.dumps(item) for item in json_TLE]

        with h5py.File(file_path, 'a') as file:
            # get the current date and format it into 'yyyymmdd' format
            current_date = datetime.now()
            formatted_date = current_date.strftime('%Y%m%d')
            # if the group named with formatted_date does not exist, create a new group with the name
            # formatted_date, and write the obtained TLE data into the group.
            if file.get(formatted_date) is None:
                file.create_group(formatted_date)
                file[formatted_date].create_dataset(formatted_date + "-2LE", data=TLE)
                file[formatted_date].create_dataset(formatted_date + "-json", data=json_TLE)
                print("\t\t\t" + constellation_name + '\'s TLE data download was successful on ' + formatted_date + ' !')
            else:
                pass
    except requests.exceptions.RequestException as e:
        print(f"\t\t\tWhen downloading TLE data, an error occurred in the web page request: {e}")
download_TLE_data('Starlink')

In [ ]:
'''get constellation'''
'''

Author : yunanhou

Date : 2023/12/09

Function : Generate and initialize constellation using TLE data

'''
import src.constellation_generation.by_TLE.get_satellite_position as GET_SATELLITE_POSITION
import src.TLE_constellation.constellation_entity.constellation as CONSTELLATION
import src.constellation_generation.by_TLE.download_TLE_data as DOWNLOAD_TLE_DATA
import src.constellation_generation.by_TLE.satellite_to_shell_mapping as SATELLITE_TO_SHELL_MAPPING
import src.constellation_generation.by_TLE.satellite_to_orbit_mapping as SATELLITE_TO_ORBIT_MAPPING
from datetime import datetime, timedelta
import os
import h5py

# Parameters:
# dT : the timeslot, and the timeslot t is calculated from 1
# constellation_name : the name of the constellation to be generated, used to read the TLE data file
def constellation_configuration(dT , constellation_name):
    # download TLE data for the current day
    DOWNLOAD_TLE_DATA.download_TLE_data(constellation_name)
    # establish the correspondence between satellites and shells
    shells = SATELLITE_TO_SHELL_MAPPING.satellite_to_shell_mapping(constellation_name)
    # establish the correspondence between satellites and orbits
    SATELLITE_TO_ORBIT_MAPPING.satellite_to_orbit_mapping(shells)

    # at this point in execution, the mapping relationship between shell, orbit and satellite has been established in
    # shells, that is, the constellation initialization function has been completed
    constellation = CONSTELLATION.constellation(constellation_name , len(shells) , shells)

    # calculate and save the longitude, latitude and altitude information of different satellites according to shell
    # and timeslot

    # determine whether the .h5 file of the delay and satellite position data of the current constellation exists. If
    # it exists, delete the file and create an empty .h5 file. If it does not exist, directly create an empty .h5 file.
    file_path = "data/TLE_constellation/" + constellation_name + ".h5"
    if os.path.exists(file_path):
        # if the .h5 file exists, delete the file
        os.remove(file_path)
    # create new empty .h5 file
    with h5py.File(file_path, 'w') as file:
        # create position group
        position = file.create_group('position')
        # create multiple shell subgroups within the position group. For example, the shell1 subgroup represents the
        # first layer of shells, the shell2 subgroup represents the second layer of shells, etc.
        for count in range(1, constellation.number_of_shells + 1, 1):
            position.create_group('shell' + str(count))

    for count in range(1, constellation.number_of_shells + 1, 1):
        # taking dT as the time interval, calculate the longitude, latitude and altitude of each satellite
        orbit_period = constellation.shells[count-1].orbit_cycle
        moments = []
        start_datetime = datetime.now()
        end_datetime = start_datetime + timedelta(seconds=orbit_period)
        while start_datetime < end_datetime:
            moments.append((start_datetime.year, start_datetime.month, start_datetime.day,
                            start_datetime.hour, start_datetime.minute, start_datetime.second))
            start_datetime += timedelta(seconds=dT)
        moments.append((end_datetime.year, end_datetime.month, end_datetime.day,
                        end_datetime.hour, end_datetime.minute, end_datetime.second))

        # the id number of satellite
        satellite_id = 1
        for satellite in constellation.shells[count-1].satellites:
            satellite.id = satellite_id
            satellite_id = satellite_id + 1
            TLE_2LE = []
            TLE_2LE.append(satellite.tle_2le[0])
            TLE_2LE.append(satellite.tle_2le[1])
            for moment in moments:
                longitude_latitude_altitude = GET_SATELLITE_POSITION.get_satellite_position(
                    TLE_2LE, moment[0], moment[1], moment[2], moment[3], moment[4], moment[5])
                satellite.longitude.append(longitude_latitude_altitude[0][0])
                satellite.latitude.append(longitude_latitude_altitude[0][1])
                satellite.altitude.append(longitude_latitude_altitude[0][2])

        for tt in range(1, len(moments)+1, 1):
            satellite_position = []
            for sat in constellation.shells[count-1].satellites:
                satellite_position.append(
                    [str(sat.longitude[tt - 1]), str(sat.latitude[tt - 1]), str(sat.altitude[tt - 1])])
            with h5py.File(file_path, 'a') as file:
                # access the existing first-level subgroup position group
                position = file['position']
                # access the existing secondary subgroup 'shell'+str(count) subgroup
                current_shell_group = position['shell' + str(count)]
                # create a new dataset in the current_shell_group subgroup
                current_shell_group.create_dataset('timeslot' + str(tt), data=satellite_position)

    return constellation


In [ ]:
'''cal topo'''
import numpy as np
from functools import cmp_to_key
import random
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import InterestingColorfulColor as ICC
import numpy as np
import json
import networkx as nx
from shapely.wkb import dumps as b_dumps, loads as b_loads

LIGHTSPEED=299792458
def get_arc(p0, p1, r=6371000+540000): # p0=[lon,lat]
	z0 = r*np.sin(np.radians(p0[1]))
	x0 = r*np.cos(np.radians(p0[1]))*np.cos(np.radians(p0[0]))
	y0 = r*np.cos(np.radians(p0[1]))*np.sin(np.radians(p0[0]))
	z1 = r*np.sin(np.radians(p1[1]))
	x1 = r*np.cos(np.radians(p1[1]))*np.cos(np.radians(p1[0]))
	y1 = r*np.cos(np.radians(p1[1]))*np.sin(np.radians(p1[0]))
	theta = np.arccos(np.dot((x0,y0,z0),(x1,y1,z1))/(r*r))
	return theta * r
def load_topo(orbit_sats,sats):
    
    have_connected={}
    #intra
    for satellites in orbit_sats:
        visit={}
        for sid in satellites:
            if sid not in have_connected:
                have_connected[sid]={'u':-1,'d':-1,'l':-1,'r':-1}
                visit[sid]=0
        while 1:
            order_dis=[]
            for dsid in satellites:
                if have_connected[dsid]['d']!=-1 and have_connected[dsid]['u']!=-1:continue
                if dsid==sid:continue
                else:
                    if sats[dsid][3]>sats[sid][3]:
                        order_dis.append((dsid,get_arc([sats[sid][2],sats[sid][3]],[sats[dsid][2],sats[dsid][3]]))
            order_dis=sorted(order_dis,key=lambda x: x[1])
            for ss in order_dis:
                if ss[0] not in have_connected:
                    have_connected[ss[0]]={'u':-1,'d':-1,'l':-1,'r':-1}
                if have_connected[sid]['d']!=-1 and have_connected[sid]['u']!=-1:break
                elif sats[ss[0]][3]>sats[sid][3] and have_connected[sid]['u']==-1 and have_connected[ss[0]]['d']==-1:
                    have_connected[sid]['u']=ss[0]
                    have_connected[ss[0]]['d']=sid
                elif sats[ss[0]][3]<sats[sid][3] and have_connected[sid]['d']==-1 and have_connected[ss[0]]['u']==-1:
                    have_connected[sid]['d']=ss[0]
                    have_connected[ss[0]]['u']=sid
            if have_connected[sid]['d']==-1:
                have_connected[sid]['d']=order_dis[0][0]
#                 for ss in order_dis:
#                     if have_connected[ss[0]]['d']!=-1 and have_connected[ss[0]]['u']!=-1:continue
#                     have_connected[sid]['d']=ss[0]
#                     if have_connected[ss[0]]['d']==-1:have_connected[ss[0]]['d']=sid
#                     else: have_connected[ss[0]]['u']=sid
            if have_connected[sid]['u']==-1:
                have_connected[sid]['u']=order_dis[0][0]
#                 for ss in order_dis:
#                     if have_connected[ss[0]]['d']!=-1 and have_connected[ss[0]]['u']!=-1:continue
#                     have_connected[sid]['d']=ss[0]
#                     if have_connected[ss[0]]['d']==-1:have_connected[ss[0]]['d']=sid
#                     else: have_connected[ss[0]]['u']=sid
    #inter
    for oid in range(len(orbit_sats)):
        loid=oid-1 if oid-1>=0 else len(orbit_sats)-1
        roid=oid+1 if oid+1<len(orbit_sats) else 0
        for sid in orbit_sats[oid]:
            if have_connected[sid]['l']!=-1 and have_connected[sid]['r']!=-1:continue
            #connect left satellites
            order_dis=[]
            for dsid in orbit_sats[loid]:
                order_dis.append((dsid,get_arc([sats[sid][2],sats[sid][3]],[sats[dsid][2],sats[dsid][3]])))
            order_dis=sorted(order_dis,key=lambda x: x[1])
            if have_connected[sid]['l']==-1:
                have_connected[sid]['l']=order_dis[0][0]
            #connect right satellites
            order_dis=[]
            for dsid in orbit_sats[roid]:
                order_dis.append((dsid,get_arc([sats[sid][2],sats[sid][3]],[sats[dsid][2],sats[dsid][3]])))
            order_dis=sorted(order_dis,key=lambda x: x[1])
            if have_connected[sid]['r']==-1:
                have_connected[sid]['r']=order_dis[0][0]
    with open('/home/chenwei/starveri/StarPerf_Simulator-release-v2.0/data/TLE_constellation/topo.txt','w')as f:
        f.write(str(have_connected))



def get_data():
    sats={}
    with h5py.File('/home/chenwei/starveri/StarPerf_Simulator-release-v2.0/data/TLE_constellation/Starlink0.h5', 'r') as file:    
        for sid in file['position']['timeslot0']:
            sat=(eval(sid[0]),eval(sid[1]),eval(sid[2]),eval(sid[3]))
            sats[eval(sid[0])]=sat
    orbits_sats=[]
    with open('/home/chenwei/starveri/StarPerf_Simulator-release-v2.0/data/TLE_constellation/Starlink_orbits.txt', 'r') as f:
        orbits_sats=eval(f.readline())
    return orbits_sats,sats
orbit_sats,sats=get_data()
load_topo(orbit_sats,sats)
def build_topo(t,sat):
    GRAPH_ISL=nx.Graph()
    topo=eval(open('/home/chenwei/starveri/StarPerf_Simulator-release-v2.0/data/TLE_constellation/topo.txt', 'r').readline())
    for satId,v in topo.items():
        for k,neighborSatId in v.items():
            dis=get_arc([sat[satId][2],sat[satId][3]],[sat[neighborSatId][2],sat[neighborSatId][3]])
            GRAPH_ISL.add_edge(satId, neighborSatId,weight=dis/LIGHTSPEED)
    return GRAPH_ISL
graph_isl=build_topo(0,sats)
print(nx.dijkstra_path_length(graph_isl,549,988),nx.dijkstra_path(graph_isl,549,988))

In [ ]:
'''draw topo'''
def ana_loc(graph_isl,orbits_sats):
    topo=eval(open('/home/chenwei/starveri/StarPerf_Simulator-release-v2.0/data/TLE_constellation/topo.txt', 'r').readline())
    intra_dis=[[]for _ in range(72)]
    for oid,ss in enumerate(orbits_sats):
        for i in range(len(ss)):
            intra_dis[oid].append(get_arc([sats[ss[i]][1],sats[ss[i]][2]],[sats[ss[i+1]][1],sats[ss[i+1]][2]])/3e8)
#             intra_dis[oid].append(abs(sats[ss[i]][2]-sats[ss[i+1]][2]))
#             intra_dis[oid].append(sats[ss[i]][3])
    for item in intra_dis:
        print(item,np.mean(item),len(item))
def draw_topo(orbit_sats,sats):
    import matplotlib as mpl
    from matplotlib import cm
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    import InterestingColorfulColor as ICC
    import json

    plt.figure(figsize=(16, 9)) # 完整地图时
    ax = plt.axes(projection=ccrs.PlateCarree() ) # central_longitude = 180     
    ax.coastlines()
    ax.add_feature(cfeature.LAND)
    ax.stock_img()
    lng,lat=[],[]
    no=5
    for sid in orbit_sats[no]:
        lng.append(sats[sid][2])
        lat.append(sats[sid][3])
        plt.text(sats[sid][2]+0.5,sats[sid][3]+0.5,str(sid))
    plt.scatter(lng, lat, marker='o', s=60, color = 'red', transform=ccrs.PlateCarree())
    lng,lat=[],[]
    
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    #plt.legend(by_label.values(), by_label.keys(), loc='lower right', fontsize = 20)
    plt.show()
draw_topo(orbit_sats,sats)


process and analyse data

In [ ]:
from tqdm import tqdm
def ana_inter_delay():
    link_delay={}
    topo=eval(open('/home/chenwei/starveri/StarPerf_Simulator-release-v2.0/data/TLE_constellation/topo.txt', 'r').readline())
    for satId,v in topo.items():
        neighborSatId=v['r']
        if satId not in link_delay:link_delay[satId]={}
        if neighborSatId not in link_delay[satId]:link_delay[satId][neighborSatId]=[]
    for slot in range(1):
        with h5py.File(f'/home/chenwei/starveri/StarPerf_Simulator-release-v2.0/data/TLE_constellation/Starlink{slot}.h5', 'r') as file:    
            for t in tqdm(range(slot*600,slot*600+600)):
                sats=[-1 for sid in range(1562)]
                for sid in file['position'][f'timeslot{t}']:
                    sat=(eval(sid[0]),eval(sid[1]),eval(sid[2]),eval(sid[3]))
                    sats[eval(sid[0])]=sat
                for satId,v in topo.items():
                    neighborSatId=v['r']
                    dis=get_arc([sats[satId][2],sats[satId][3]],[sats[neighborSatId][2],sats[neighborSatId][3]])/LIGHTSPEED
                    link_delay[satId][neighborSatId].append(dis)

    #LLC
#     for orbitId in range(72):
#         for satInOrbitId in range(22):
#             satId = orbitId * 22 + satInOrbitId
#             neighborOrbitId = (orbitId + 1) % 72
#             neighborSatInOrbitId = satInOrbitId
#             neighborSatId = neighborOrbitId * 22 + neighborSatInOrbitId
#             if satId not in link_delay:
#                 link_delay[satId]={}
#             if neighborSatId not in link_delay[satId]:
#                 link_delay[satId][neighborSatId]=[]
#     for t in range(6000):
#         satPos=load_satPos(f'/home/chenwei/CrossShellRouting/pos_data/Starlink1/Starlink1_{t}.txt')
#         for orbitId in range(72):
#             for satInOrbitId in range(22):
#                 satId = orbitId * 22 + satInOrbitId
#                 neighborOrbitId = (orbitId + 1) % 72
#                 neighborSatInOrbitId = satInOrbitId
#                 neighborSatId = neighborOrbitId * 22 + neighborSatInOrbitId
#                 dis=get_arc([satPos[satId]['lng'],satPos[satId]['lat']],[satPos[neighborSatId]['lng'],satPos[neighborSatId]['lat']])/LIGHTSPEED
#                 link_delay[satId][neighborSatId].append(dis)
    return link_delay
link_delay=ana_inter_delay()

In [ ]:
def process_link_delay(link_delay):
    delays={}
    for sid,item in link_delay.items():
        if sid not in delays:
            delays[sid]={}
        
        for neighbor,des in item.items():
            if neighbor not in delays[sid]:delays[sid][neighbor]=[]
            pre_d=-1
            pre_t=0
            for t,d in enumerate(des):
                if pre_d==-1:
                    pre_d=d
                    continue
                if abs(d-pre_d)>10e-4:
                    pre_d=d
                    delays[sid][neighbor].append(t-pre_t)
                    pre_t=t
#     for k,v in delays.items():
#         for i in v.values():
#             if i==[]:continue
#             print(max(i),min(i),np.mean(i))
#             break
#         break
    maxd=[]
    mind=[]
    meand=[]
    for k,v in delays.items():
        for i in v.values():
            if i!=[]:
                maxd.append(max(i))
                mind.append(min(i))
                meand.append(np.mean(i))
            else:
                maxd.append(600)
                mind.append(600)
                meand.append(600)
    return delays,maxd,mind,meand
delays,maxd,mind,meand=process_link_delay(link_delay)
